In [ ]:
import autograd.numpy as np
import torch
from numpy import linalg as la
from numpy import random as rnd

import pymanopt
from pymanopt.manifolds import PSDFixedRank
from pymanopt.solvers import TrustRegions


SUPPORTED_BACKENDS = ("Autograd", "Callable", "PyTorch")


def create_cost_egrad_ehess(manifold, matrix, backend):
    egrad = ehess = None

    @pymanopt.function.Autograd(manifold)
    def cost(Y):
        return np.linalg.norm(Y @ Y.T - matrix, "fro") ** 2

    return cost, egrad, ehess


backend=SUPPORTED_BACKENDS[0]
quiet=False
num_rows = 5
rank = 5
low_rank_factor = rnd.randn(num_rows, rank)
matrix = low_rank_factor @ low_rank_factor.T

manifold = PSDFixedRank(num_rows, rank)
cost, egrad, ehess = create_cost_egrad_ehess(manifold, matrix, backend)
problem = pymanopt.Problem(manifold, cost=cost, egrad=egrad, ehess=ehess)
if quiet:
    problem.verbosity = 0


solver = TrustRegions(maxiter=500, minstepsize=1e-6)
low_rank_factor_estimate = solver.solve(problem)

print("Rank of target matrix:", la.matrix_rank(matrix))
matrix_estimate = low_rank_factor_estimate @ low_rank_factor_estimate.T
print(
    "Frobenius norm error of low-rank estimate:",
    la.norm(matrix - matrix_estimate),
)